##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hyperparameter Tuning with the HParams Dashboard

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/r2/hyperparameter_tuning_with_hparams"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/r2/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

When building machine learning models, you need to choose various [hyperparameters](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)), such as the dropout rate in a layer or the learning rate. These decisions impact model metrics, such as accuracy. Therefore, an important step in the machine learning workflow is to identify the best hyperparameters for your problem, which often involves experimentation. This process is known as "Hyperparameter Optimization" or "Hyperparameter Tuning".

The HParams dashboard in TensorBoard provides several tools to help with this process of identifying the best experiment or most promising sets of hyperparameters. 

This tutorial will focus on the following steps:

1. Experiment setup and HParams summary
2. Adapt TensorFlow runs to log hyperparameters and metrics
3. Start runs and log them all under one parent directory
4. Visualize the results in TensorBoard's HParams dashboard

Note: The HParams summary APIs and dashboard UI are in a preview stage and will change over time. 

Start by installing TF 2.0 and loading the TensorBoard notebook extension:

In [0]:
!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard 

    100% |████████████████████████████████| 79.8MB 324kB/s 
    100% |████████████████████████████████| 348kB 19.2MB/s 
    100% |████████████████████████████████| 3.0MB 11.4MB/s 
    100% |████████████████████████████████| 61kB 18.2MB/s 


In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Import TensorFlow and other packages needed for the HParams dashboard:

In [0]:
import datetime
import tensorflow as tf

# Imports for the HParams plugin
from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2

Download the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and scale it:

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


## 1. Experiment setup and the HParams experiment summary

Experiment with three hyperparameters in the model:

1. Number of units in the first dense layer
2. Dropout rate in the dropout layer
3. Optimizer

List the values to try:

In [0]:
num_units_list = [16, 32]
dropout_rate_list = [0.1, 0.2] 
optimizer_list = ['adam', 'sgd'] 

Log an experiment summary. This is how the HParams dashboard knows what are the hyperparameters and metrics (API details are [here](https://github.com/tensorflow/tensorboard/blob/master/tensorboard/plugins/hparams/api.proto)):

In [0]:
def create_experiment_summary(num_units_list, dropout_rate_list, optimizer_list):
  num_units_list_val = struct_pb2.ListValue()
  num_units_list_val.extend(num_units_list)
  dropout_rate_list_val = struct_pb2.ListValue()
  dropout_rate_list_val.extend(dropout_rate_list)
  optimizer_list_val = struct_pb2.ListValue()
  optimizer_list_val.extend(optimizer_list)
  return hparams_summary.experiment_pb(
      # The hyperparameters being changed
      hparam_infos=[
          api_pb2.HParamInfo(name='num_units',
                             display_name='Number of units',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=num_units_list_val),
          api_pb2.HParamInfo(name='dropout_rate',
                             display_name='Dropout rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=dropout_rate_list_val),
          api_pb2.HParamInfo(name='optimizer',
                             display_name='Optimizer',
                             type=api_pb2.DATA_TYPE_STRING,
                             domain_discrete=optimizer_list_val)
      ],
      # The metrics being tracked
      metric_infos=[
          api_pb2.MetricInfo(
              name=api_pb2.MetricName(
                  tag='accuracy'),
              display_name='Accuracy'),
      ]
  )

exp_summary = create_experiment_summary(num_units_list, dropout_rate_list, optimizer_list)
root_logdir_writer = tf.summary.create_file_writer("logs/hparam_tuning")
with root_logdir_writer.as_default():
  tf.summary.import_event(tf.compat.v1.Event(summary=exp_summary).SerializeToString())

## 2. Adapt TensorFlow runs to log hyperparameters and metrics

The model will be quite simple: two dense layers with a dropout layer between them. The training code will look familiar, although the hyperparameters are no longer hardcoded. Instead, the hyperparameters are provided in an `hparams` dictionary and used throughout the training function:

In [0]:
def train_test_model(hparams):

  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams['num_units'], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams['dropout_rate']),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  model.compile(optimizer=hparams['optimizer'],
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

For each run, log an hparams summary with the hyperparameters and final accuracy:

In [0]:
def run(run_dir, hparams):
  writer = tf.summary.create_file_writer(run_dir)
  summary_start = hparams_summary.session_start_pb(hparams=hparams)

  with writer.as_default():
    accuracy = train_test_model(hparams)
    summary_end = hparams_summary.session_end_pb(api_pb2.STATUS_SUCCESS)
      
    tf.summary.scalar('accuracy', accuracy, step=1, description="The accuracy")
    tf.summary.import_event(tf.compat.v1.Event(summary=summary_start).SerializeToString())
    tf.summary.import_event(tf.compat.v1.Event(summary=summary_end).SerializeToString())

## 3. Start runs and log them all under one parent directory

You can now try multiple experiments, training each one with a different set of hyperparameters. 

For simplicity, try all combinations (this is called a grid search). For more complex scenarios, it might be more effective to choose each hyperparameter value randomly (this is called a random search). There are more advanced methods that can be used.

Run a few experiments, which will take a few minutes:

In [0]:
session_num = 0

for num_units in num_units_list:
  for dropout_rate in dropout_rate_list:
    for optimizer in optimizer_list:
      hparams = {'num_units': num_units, 'dropout_rate': dropout_rate, 'optimizer': optimizer}
      print('--- Running training session %d' % (session_num + 1))
      print(hparams)
      run_name = "run-%d" % session_num
      run("logs/hparam_tuning/" + run_name, hparams)
      session_num += 1


--- Running training session 1
{'num_units': 16, 'dropout_rate': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 38us/sample - loss: 0.5077 - accuracy: 0.8219
--- Running training session 2
{'num_units': 16, 'dropout_rate': 0.1, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 39us/sample - loss: 1.3829 - accuracy: 0.6366
--- Running training session 3
{'num_units': 16, 'dropout_rate': 0.2, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 39us/sample - loss: 0.5112 - accuracy: 0.8199
--- Running training session 4
{'num_units': 16, 'dropout_rate': 0.2, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 38us/sample - loss: 1.4468 - accuracy: 0.5909
--- Running training session 5
{'num_units': 32, 'dropout_rate': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 40us/sample - loss: 0.4662 - accuracy: 0.8335
--- Running training session 6
{'num_units': 32, 'dropout_rate': 0.

## 4. Visualize the results in TensorBoard's HParams plugin

The HParams dashboard can now be opened. Start TensorBoard and click on "HParams" at the top.

In [0]:
%tensorboard --logdir logs/hparam_tuning

<img class="tfo-display-only-on-site" src="images/hparams_table.png?raw=1"/>

The left pane of the dashboard provides filtering capabilities that are active across all the views in the HParams dashboard:

- Filter which hyperparameters/metrics are shown in the dashboard
- Filter which hyperparameter/metrics values are shown in the dashboard
- Filter on run status (running, success, ...)
- Sort by hyperparameter/metric in the table view
- Number of session groups to show (useful for performance when there are many experiments)


The HParams dashboard has three different views, with various useful information:

* The **Table View** lists the runs, their hyperparameters, and their metrics.
* The **Parallel Coordinates View** shows each run as a line going through an axis for each hyperparemeter and metric. Click and drag the mouse on any axis to mark a region which will highlight only the runs that pass through it. This can be useful for identifying which groups of hyperparameters are most important. The axes themselves can be re-ordered by dragging them.
* The **Scatter Plot View** shows plots comparing each hyperparameter/metric with each metric. This can help identify correlations. Click and drag to select a region in a specific plot and highlight those sessions across the other plots. 

A table row, a parallel coordinates line, and a scatter plot market can be clicked to see a plot of the metrics as a function of training steps for that session (although in this tutorial only one step is used for each run).

To further explore the capabilities of the HParams dashboard, download a set of pregenerated logs with more experiments:

In [0]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

View these logs in TensorBoard:

In [0]:
%tensorboard --logdir logs/hparam_demo

<img class="tfo-display-only-on-site" src="images/hparams_parallel_coordinates.png?raw=1"/>

You can try out the different views in the HParams dashboard. 

For example, by going to the parallel coordinates view and clicking and dragging on the accuracy axis, you can select the runs with the highest accuracy. As these runs pass through 'adam' in the optimizer axis, you can conclude that 'adam' performed better than 'sgd' on these experiments.